# Storage/Waste Bin

### Converting time to space

The curve for voltage over time:
$$
\begin{align*}
    \nu &= b + a_2 \sin^2 (b_2 \pi t + c_2 \pi) \\
    V   &= a \sin^2(\nu \pi t + c\pi) + d + e t 
\end{align*}
$$

> $a$ is in volts so no unit change

> $b$ is in $1/t$:
  $$
    \begin{align*}
        b \pi t = (2 \pi / \lambda) x 
        & \implies x = b \lambda t /2 \\
        & \implies t = 2 x / (b \lambda)
    \end{align*}
  $$

> $c$ is dimensionsless so no change

> $d$ is in volts so no unit change

> $e$ is in units of $1/t$:
  $$
    \begin{align*}
        e t \implies e 2 x / (b \lambda) \implies E = 2e / (b\lambda) 
    \end{align*}
  $$
  
> $a_2$ is in units of $1/t$:
  $$
    \begin{align*}
        a_2 t \implies a_2 2 x / (b \lambda) \implies A_2 = 2 a_2 / (b\lambda)
    \end{align*}
  $$
 
> $b_2$ is in units of $1/t$:
  $$
    \begin{align*}
        b_2 t \implies b_2 2 x / (b \lambda) \implies A_2 = 2 b_2 / (b\lambda)
    \end{align*}
  $$

> $c_2$ is dimensionless so no unit change

In [13]:
def FuncCoeffTToX(a, b, c, d, e, a2, b2, c2, lam=632):
    A = a
    B = 2/lam
    C = c
    D = d
    E = e * 2 / (b * lam)
    A2 = a2 * 2 / (b * lam)
    B2 = b2 * 2 / (b * lam)
    C2 = c2
    return A, B, C, D, E, A2, B2, C2

@np.vectorize
def LinCoeffTToX(a, b, lam=632):
    A = a * 2 / (b * lam)
    B = b
    return A, B

@np.vectorize
def TtoX(t, b, lam=632):
    x = (b*lam*t)/2
    return x

In [14]:
# Commence conversion
PV['Displacement (nm)'] = TtoX(PV['Time Zeroed (s)'], b1)
DV['Displacement (nm)'] = TtoX(DV['Time Zeroed (s)'], b1)

PVDisp, PVDisp2 = PV['Displacement (nm)'][Cut0Pos:Cut1Pos], PV['Displacement (nm)'][Cut1Pos:Cut2Pos]
DVDisp, DVDisp2 = DV['Displacement (nm)'][Cut0Pos:Cut1Pos], DV['Displacement (nm)'][Cut1Pos:Cut2Pos]

# Convert coefficients

PVPopt = LinCoeffTToX(*PVpopt)
PVPopt2 = LinCoeffTToX(*PVpopt2)


DVPopt = FuncCoeffTToX(*DVpopt, lam=632)
DVPopt2 = FuncCoeffTToX(*DVpopt2, lam=632)

In [25]:
# Plot it out
#============

# Plot it out 
fig, ax = plt.subplots(2, 1, figsize = (8, 10))

# Voltage applied to piezo  per time
# PV.plot(x = 'Displacement (nm)', y = 'Voltage (mV)', ax = ax[0], legend=False,
#         title = 'Voltage applied to piezo crystal',
#         xlabel = 'nm',
#         ylabel = 'mV')

ax[0].plot(PVDisp, LinFunc(PVDisp, *PVpopt), c='orange', zorder=3,
           label = r'Fit: $V = ({0:.4g}) x  {1:+.4g}$'.format(*PVPopt))

ax[0].plot(PVDisp2, LinFunc(PVDisp2, *PVpopt2), c='red', zorder=3,
           label = r'Fit: $V = ({0:.4g}) x  {1:+.4g}$'.format(*PVPopt2))

# ax[0].plot(PV['Displacement (nm)'], PVHan, c='black',
#            label = 'Hanning over {0} samples'.format(HanIntPV))

ax[0].legend()
ax[0].grid()

# Voltage output of photodetector per time
# DV.plot(x = 'Displacement (nm)', y = 'Voltage (V)', legend=False, ax = ax[1],
#         title = 'Voltage output of photodetector',
#         xlabel = 'nm',
#         ylabel = 'V',
#         label = 'Data')

ax[1].plot(DVDisp, DVFunc(DVDisp, *DVPopt), c='orange', zorder=3,
          label = 
               r'Fit: $V = {0:.4g} \sin^2(\nu \pi x {2:+.4g} \pi) {3:+.4g} {4:+.4g}x$' '\n' 
               r'Where $\nu= {1:.4g} {5:+.4g} \sin^2 ({6:.4g} \pi x {7:+.4g}\pi)$'
               .format(*DVPopt))

ax[1].plot(DVDisp2, DVFunc(DVDisp2, *DVPopt2), c='red', zorder=3,
          label = 
               r'Fit: $V = {0:.4g} \sin^2(\nu \pi x {2:+.4g} \pi) {3:+.4g} {4:+.4g}x$' '\n'
               r'Where $\nu= {1:.4g} {5:+.4g} \sin^2 ({6:.4g} \pi x {7:+.4g}\pi)$'
               .format(*DVPopt2))

# ax[1].plot(DV['Displacement (nm)'], DVHan, c='black', zorder=2,
#            label = 'Hanning over {0} samples'.format(HanIntDV))

ax[1].set_ylim(np.min(DV['Voltage (V)'])*1.4)
ax[1].legend(loc='lower left')
ax[1].grid()

<Figure size 576x720 with 2 Axes>

### Graphing the voltage output of the photodetector against voltage applied to 

In [16]:

plt.plot(PV['Voltage (mV)'], DV['Intensity'])

plt.plot(PVHan, DVHanIntn, c='black')


<Figure size 432x288 with 1 Axes>

## Figuring out the displacement of piezo crystal per volt

Wavelength of the laser: $\lambda = 632$ nm

Frequency of the photodetector: $\nu_D = 3969.668$ Hz or $\lambda / s$

This means that the piezo is moving at $v_P = \nu_D / 2$ $\lambda/s$

For that period, the voltage applied to the piezo is increasing at $1.7 \times 10^4$ mV/s = $17$ V/s



In [17]:
T1 = np.arange(-5*np.pi, 5*np.pi, 0.001)
T2 = np.unwrap(T1)
print(T1, T2)

[-15.70796327 -15.70696327 -15.70596327 ...  15.70503673  15.70603673
  15.70703673] [-15.70796327 -15.70696327 -15.70596327 ...  15.70503673  15.70603673
  15.70703673]


In [18]:
plt.plot(T1, T2)

<Figure size 432x288 with 1 Axes>